Changes we decided on: 

1. Only the new transcripts.

2. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install lda
!pip install nltk
!pip install numba
from numba import jit
import pandas as pd
from keras.preprocessing.text import Tokenizer

In [ ]:
import nltk
from nltk.corpus import stopwords
import re
nltk.download('stopwords')
def preprocess(sentence):
  stop_words=stopwords.words('english')
  cleaned_sentence_list = []
  for word in sentence.split():
    removed_word = re.sub(r'[^a-zA-Z]','',word).lower()
    if removed_word not in stop_words:
       cleaned_sentence_list.append(removed_word)
  cleaned_sentence = " ".join(cleaned_sentence_list)
  return cleaned_sentence

def merge_JsonFiles(filename, custom = "/NLPAI/Data/Phase2Data/"):
    result = list()
    for f1 in filename:
      f1 = '/content/drive/Shareddrives/Annota' + custom + f1
      with open(f1, 'r') as infile:
        result.extend(json.load(infile))

    return result

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import json

rq_txt = "/content/drive/Shareddrives/Annota/NLPAI/Data/Phase2Misc/rq_list.txt"
with open(rq_txt, "r") as f:
  rqs = f.read()
def text_to_list(text):
  text = text[1:-1]
  text = re.split("',", text)
  text = [tx.replace("'", "").strip() for tx in text]
  return text
rqs = text_to_list(rqs)

rq_transcripts = "/content/drive/Shareddrives/Annota/NLPAI/Data/Phase2Misc/rq_map.json"
with open(rq_transcripts, "r") as f:
  rq_map = json.load(f)

exclude = ["Chris", "Marketing Chair", "Jane Doe"]
rq_map = {name: rq_map[name] for name in rq_map.keys() if name not in exclude}

is_graded = "/content/drive/Shareddrives/Annota/NLPAI/Data/Phase2Misc/graded.json"
with open(is_graded, "r") as f:
  graded = json.load(f)

graded = [name for name in graded if name not in exclude]
graded

['Xiulan Al-Hashim',
 'Guanyu Sun',
 'Emma McCarthy',
 'Amelia Winton',
 'Colson Foster',
 'Edith Stacy',
 'Jay',
 'Kalyana Martini',
 'Aaron Rush',
 'Alice',
 'Oda Power',
 'Sierra Snow',
 'Uma Rose',
 'Jane Smith',
 'David Ayers',
 'Victor Clayton',
 'Bell Maria',
 'Kate Ollison',
 'Thomas Cao',
 'Katsu Sun',
 'Jaya Power']

In [ ]:
def collect_data(rq):

  relevant = [name for name in rq_map if rq in rq_map[name]] ##We got the relevant names
  rel_grad = [t + ".json" for t in relevant if t in graded]
  rel_ungrad = list(set(relevant) - set(rel_grad))
  rel_ungrad = [name + ".json" for name in rel_ungrad]
  train_data = merge_JsonFiles(rel_ungrad) ##We have a list of items. We want to apply clean to each of their text, and if we return an empty string, remove this string.
  if len(rel_grad) == 0:
    return -1, -1
  test_data = merge_JsonFiles(rel_grad)
  train_index = 0
  while train_index < len(train_data):
    train_data[train_index]["text"] = preprocess(train_data[train_index]["text"]).strip().replace("  ", " ")
    text = (train_data[train_index]["text"])
    while "  " in text:
      text = text[:text.index("  ")] + text[text.index("  ") + 1: ]
    text = str(text)
    train_data[train_index]["text"] = text
    assert "  " not in train_data[train_index]["text"], train_data[train_index]["text"] 
    if(train_data[train_index]["text"].strip() == ""):
      train_data = train_data[:train_index] + train_data[train_index + 1:]
    else:
      train_index += 1
  
  test_index = 0
  while test_index < len(test_data):
    test_data[test_index]["text"] = preprocess(test_data[test_index]["text"]).strip()
    text = (test_data[test_index]["text"])
    while "  " in text:
      text = text[:text.index("  ")] + text[text.index("  ") + 1: ]
    test_data[test_index]["text"] = str(text)
    assert "  " not in test_data[test_index]["text"], test_data[test_index]["text"] 
    if(test_data[test_index]["text"].strip() == ""):
      test_data = test_data[:test_index] + test_data[test_index + 1:]
    else:
      test_index+=1

  return train_data, test_data  

In [ ]:
def text_preprocess(train_data, test_data):
  #train_data_text = [preprocess(data["text"]) for data in train_data]
  #test_data_text = [preprocess(data["text"]) for data in test_data]
  train_data_text = [data["text"] for data in train_data]
  test_data_text = [data["text"] for data in test_data]
  train_data_text = [text for text in train_data_text if text.strip() != ""]
  test_data_text = [text for text in test_data_text if text.strip() != ""]
  return train_data_text, test_data_text

In [ ]:
def tokenize(train_docs, test_docs):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(train_docs)
  train_vectors = tokenizer.texts_to_matrix(train_docs, mode='count')
  test_tokenizer = Tokenizer()
  test_tokenizer.fit_on_texts(test_docs)
  test_vectors = test_tokenizer.texts_to_matrix(test_docs, mode='count')
  return train_vectors[:, 1:], test_vectors[:, 1:], tokenizer, test_tokenizer

In [ ]:
import string
def get_overlap(sent1,sent2):
  stop_words = set(stopwords.words('english'))
  PUNCT_REMOVE = string.punctuation

  sent1=sent1.translate(str.maketrans('', '', PUNCT_REMOVE))
  sent2=sent2.translate(str.maketrans('', '', PUNCT_REMOVE))
  s1=nltk.word_tokenize(sent1)
  
  s2=nltk.word_tokenize(sent2)
  s1=[t for t in s1 if t.lower() not in stop_words]
  s2=[t for t in s2 if t.lower() not in stop_words]
  if len(s1)>=2:
    b1=nltk.bigrams(s1)
    s1=set([t for t in b1])
  else:
    s1=set(s1)
  if len(s2)>=2:
    b2=nltk.bigrams(s2)
    s2=set([t for t in b2])
  else:
    s2=set(s2)
  return len(s1 & s2)

In [ ]:
import numpy as np
import random
import lda
import lda.datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn import preprocessing
from collections import defaultdict
def has_rq(rq, ann_list):
  rq = rq.strip()
  for ann in ann_list:
    if ann["ResearchQuestionId"] == rq:
      return True
  return False

rel_map_dict = {"NOT_RELEVANT": 0, "RELEVANT": 1, "REQUIRED": 1}


def rel_map(relevance):
  if relevance in rel_map_dict.keys():
    return rel_map_dict[relevance]
  elif relevance == "UNSURE":
    return 1

def lda_df_labels(rq, train_vectors, test_vectors, train_data, test_data, train_docs, test_docs, tokenizer,test_tokenizer, n_top_words = 8):
  
  X = np.array(train_vectors).astype(int)
  vocab = list(tokenizer.word_index.keys())
  model = lda.LDA(n_topics = 10, n_iter = 1500, random_state = 1)
  model.fit(X)
  topic_word = model.topic_word_
  doc_topic = model.doc_topic_
  
  X = np.array(test_vectors).astype(int)
  test_vocab = list(test_tokenizer.word_index.keys())
  print("There are ", len(set(vocab) - (set(vocab) | set(test_vocab))), "words we don't know of, like: ", set(vocab) - (set(vocab) | set(test_vocab)))
  test_model = lda.LDA(n_topics = 10, n_iter = 1500, random_state = 1)
  test_model.fit(X)
  test_topic_word = test_model.topic_word_
  test_doc_topic = test_model.doc_topic_
  
  train_label_dict = dict()
  test_label_dict = dict()
  train_annotators = set()
  test_annotators = set()
  train_df_data = []
  test_df_data = []
  
  ##For each sentence, store the sentence. 
  
  for item in train_data:
    if has_rq(rq, item["annotations"]):
      rq_item = item
      rq_item["annotations"] = [ann for ann in rq_item["annotations"] if ann["ResearchQuestionId"] == rq]
      rq_item["count"] = len(rq_item["annotations"])
      train_df_data.append(rq_item)
    
    else:
      continue

    for annotation in rq_item["annotations"]:
      train_annotators.add(annotation["id"])

  train_annotators = sorted(list(train_annotators))

  for rq_item in train_df_data:
    text = rq_item["text"]
    
    if text not in train_label_dict.keys():
      train_label_dict[text] = []
      for ann in train_annotators:
        present = False
        for itm in rq_item["annotations"]:
          if ann == itm["id"]:
            present = True
        if present:
          train_label_dict[text].append(1.0)
        else:
          train_label_dict[text].append(0.0)
  
  for item in test_data:
    if "grader_annotations" in item.keys():
      test_df_data.append(item)
    #if has_rq(rq, item["annotations"]):
    #  rq_item = item
    #  rq_item["annotations"] = [ann for ann in rq_item["annotations"] if ann["ResearchQuestionId"] == rq]
    #  rq_item["count"] = len(rq_item["annotations"])
    #  test_df_data.append(rq_item)
    
    #else:
    #  continue

    for annotation in rq_item["annotations"]:
      test_annotators.add(annotation["id"])

  test_annotators = sorted(list(test_annotators))
  
  for rq_item in test_df_data:
    text = rq_item["text"]
    
    if text not in test_label_dict.keys():

      test_label_dict[text] = []    
      try:
        test_label_dict[text] = (rel_map(rq_item["grader_annotations"][rq]))
      except:
        print("rq: ", rq)
        print("text: ", text)
        print(rq_item["grader_annotations"])
        assert False

  topics_labels_train = defaultdict(list)
  topics_labels_train["text"] = list(train_docs)

  topics_labels_test = defaultdict(list)
  topics_labels_test["text"] = list(test_label_dict.keys())

  for txt in train_docs:
    index = train_docs.index(txt)
    for j, distribution in enumerate(doc_topic[index]):
      topics_labels_train["topic" + str(j)].append(distribution)
  
  for ann_index in range(len(train_annotators)):
    ann = train_annotators[ann_index]
    try:
      topics_labels_train[ann] = [train_label_dict[txt][ann_index] if txt in train_label_dict.keys() else 0.0 for txt in train_docs]
    except:
      for txt in train_docs:
        if txt not in train_label_dict.keys():
          assert False
  
  for txt in test_label_dict.keys():
    index = test_docs.index(txt)
    for j, distribution in enumerate(test_doc_topic[index]):
      topics_labels_test["topic" + str(j)].append(distribution)

  #for ann_index in range(len(test_annotators)):
  #  ann = test_annotators[ann_index]
  #  topics_labels_test[ann] = [test_label_dict[txt][ann_index] if txt in test_label_dict.keys() else 0.0 for txt in test_docs]
  
  topics_labels_test["grader"] = [test_label_dict[txt] for txt in test_label_dict.keys()]
  
  ##I don't know how to match the cleaned sentences and the direct transcript that we had before....
  
  
  the_len = -1
  for key in topics_labels_test.keys():
    if the_len == -1:
      the_len = len(topics_labels_test[key])
    assert len(topics_labels_test[key]) == the_len
  
  topics_labels_test_df = pd.DataFrame(topics_labels_test)
  topics_labels_train_df = pd.DataFrame(topics_labels_train)
  return topics_labels_train_df, topics_labels_test_df

In [ ]:
def iosplit(train_df, test_df, aggregate = True):
  train_df_columns = list(train_df.columns)
  test_df_columns = list(test_df.columns)
  
  inputs = [feature for feature in train_df_columns if "topic" in feature]
  inp_ind = [feature_index for feature_index in range(len(train_df_columns)) if "topic" in train_df_columns[feature_index]]
  anns = [feature for feature in train_df_columns if "ann" in feature]
  ann_ind = [feature_index for feature_index in range(len(train_df_columns)) if "ann" in train_df_columns[feature_index]]
  test_inputs = [feature for feature in test_df_columns if "topic" in feature]
  test_inp_ind = [feature_index for feature_index in range(len(test_df_columns)) if "topic" in test_df_columns[feature_index]]
  
  train_inp_df = train_df[inputs]
  train_ann_df = train_df[anns]

  test_inp_df = test_df[inputs]
  test_grad_df = test_df[["grader"]]
  
  X_train = train_inp_df.to_numpy()
  y_train = train_ann_df.to_numpy()
  X_test = test_inp_df.to_numpy()
  y_test = test_grad_df.to_numpy()
  
  
  majority_train = np.sum(y_train, axis = 1)

  stat_out = np.zeros(majority_train.shape)
  stat_out[majority_train >= 2] = 1
  stat_out[majority_train >= 4] = 2

  print("X_train.shape: ", np.shape(X_train))
  print("y_train.shape: ", np.shape(y_train))
  print("stat_out.shape: ", np.shape(stat_out))
  print("X_test.shape: ", np.shape(X_test))
  print("y_test.shape: ", np.shape(y_test))
  if aggregate:
    return X_train, stat_out, X_test, y_test
  
  else:
    return X_train, stat_out, y_train, X_test, y_test

In [ ]:
def init_sens(y_train, majority_train):
  alpha = np.sum(np.matmul(y_train, majority_train), axis = 0)
  return alpha

In [ ]:
def init_spec(y_train, majority_train):
  beta = np.sum(np.matmul(1 - y_train, 1 - majority_train), axis = 0)
  return beta

In [ ]:
import tqdm
from tqdm import tqdm
from numpy.linalg import pinv
import numba

def binary_em(X_train, y_train, majority_train, classifier):
  alpha = np.sum(np.matmul(y_train.T, majority_train), axis = 0)/np.sum(majority_train)
  beta = np.sum(np.matmul(1 - y_train.T, 1 - majority_train), axis = 0)/np.sum(1-majority_train)
  mu = np.mean(y_train, axis = 0)
  a = np.prod(np.power(alpha, y_train), axis = 1) * np.prod(np.power(1 - alpha, 1 - y_train), axis = 1) 
  b = np.prod(np.power(beta, 1 - y_train), axis = 1) * np.prod(np.power(1 - beta, y_train), axis = 1)
  classifier.fit(X_train, majority_train)
  p = classifier.predict_proba(X_train)[:, 0]
  em_steps = 15
  grad_steps = 300
  lr = 0.01
  for e in tqdm(range(em_steps)):
    
    ##Expectation:
    a = np.prod(np.power(alpha, y_train), axis = 1) * np.prod(np.power(1 - alpha, 1 - y_train), axis = 1) 
    b = np.prod(np.power(beta, 1 - y_train), axis = 1) * np.prod(np.power(1 - beta, y_train), axis = 1)
    p = classifier.predict_proba(X_train)[:,0]
    mu = a * p / (a * p + b * (1 - p))
    E = np.sum(mu * np.log(p*a) + (1 - mu) * np.log((1 - p) * b))
    
    ##Maximization:

    alpha = np.matmul(y_train.T, mu).T / np.sum(mu)
    beta = np.matmul((1 - y_train.T), (1 - mu)).T / np.sum(1 - mu)

    ##Learning Weights:

    weights = classifier.coef_[0]
    for grad_steps in range(grad_steps):
      pred = classifier.predict_proba(X_train)[:, 0]
      grad = np.sum(np.multiply((mu - pred).reshape(-1,1), X_train), axis = 0)
      
      #Gradient Descent:
      ##weights -= lr * grad
      
      ##Newton's Method:
      hess_shape = (np.shape(X_train))[1]
      hess_shape = (hess_shape, hess_shape)
      hessian = np.zeros(hess_shape)
      for i in range(len(X_train)):
          hessian -= pred[i] * (1 - pred[i]) * np.outer(X_train[i], X_train[i])
      hessian_inv = pinv(hessian)
      weights = weights - lr * np.matmul(hessian_inv, grad)
      classifier.coef_[0] = weights
  
  return classifier

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from numpy.core.numeric import binary_repr
rq_accs = []
rq_trains = []
for rq_index in tqdm(range(len(rqs))): 
  rq = rqs[rq_index]
  print(rq.upper())

  train_data, test_data = collect_data(rq) ##Collect Data will merge all JsonFiles with this rq into training data and testing data based on whether there are any grader annotations

  if(isinstance(train_data, int) and isinstance(test_data, int)):
    print("Didn't find grading for", rq)
    continue
  
  train_docs, test_docs = text_preprocess(train_data, test_data) ##Will convert to text and preprocess both training and testing data.
  
  train_vectors, test_vectors, tokenizer, test_tokenizer = tokenize(train_docs, test_docs) ##Will tokenize them and return the corresponding matrices

  train_df, test_df = lda_df_labels(rq, train_vectors, test_vectors, train_data, test_data, train_docs, test_docs, tokenizer, test_tokenizer) ##Will fit the train/test vectors independently to topics, and model their distributions, then collect labels, and convert all of them into the finished dataframe. For now, we will exclude the annotator labels though we collect them, to allow LogisticRegression to be tested on them first.

  X_train, majority_train, y_train, X_test, y_test = iosplit(train_df, test_df, aggregate = False) ##Will split each dataframe into input and output for the model

  if list(y_test).count(1) == 0 or list(y_test).count(0) == 0:
    print("Data is imbalanced, skipping.")
    continue
  
  classifier = LogisticRegression()
  
  majority_train = majority_train.reshape(-1, 1)
  #majority_test = np.sum(y_test, axis = 1)

  #stat_out = np.zeros(majority_train.shape)
  #stat_out[majority_train >= 2] = 1
  #stat_out[majority_train >= 4] = 2

  classifier = binary_em(X_train, y_train, majority_train, classifier)
  for row in X_test:
    for col in row:
      assert not isinstance(col, str), col
  rq_accs.append(f1_score(y_test, classifier.predict(X_test), average = "macro"))
  rq_trains.append(f1_score(majority_train, classifier.predict(X_train), average = "macro"))
  
  print(rq, ":", rq_accs[-1])

  0%|          | 0/13 [00:00<?, ?it/s]

ARTICULATING PROBLEMS


INFO:lda:n_documents: 4261
INFO:lda:vocab_size: 3251
INFO:lda:n_words: 26737
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -292173
INFO:lda:<10> log likelihood: -210040
INFO:lda:<20> log likelihood: -205106
INFO:lda:<30> log likelihood: -203420
INFO:lda:<40> log likelihood: -202216
INFO:lda:<50> log likelihood: -201446
INFO:lda:<60> log likelihood: -200826
INFO:lda:<70> log likelihood: -200270
INFO:lda:<80> log likelihood: -199750
INFO:lda:<90> log likelihood: -199323
INFO:lda:<100> log likelihood: -198941
INFO:lda:<110> log likelihood: -198850
INFO:lda:<120> log likelihood: -198589
INFO:lda:<130> log likelihood: -198504
INFO:lda:<140> log likelihood: -198232
INFO:lda:<150> log likelihood: -197931
INFO:lda:<160> log likelihood: -198006
INFO:lda:<170> log likelihood: -197617
INFO:lda:<180> log likelihood: -197666
INFO:lda:<190> log likelihood: -197466
INFO:lda:<200> log likelihood: -197800
INFO:lda:<210> log likelihood: -197364
INFO:lda:<220> log likelihood: -

There are  0 words we don't know of, like:  set()


INFO:lda:<50> log likelihood: -13778
INFO:lda:<60> log likelihood: -13753
INFO:lda:<70> log likelihood: -13784
INFO:lda:<80> log likelihood: -13821
INFO:lda:<90> log likelihood: -13735
INFO:lda:<100> log likelihood: -13737
INFO:lda:<110> log likelihood: -13773
INFO:lda:<120> log likelihood: -13822
INFO:lda:<130> log likelihood: -13781
INFO:lda:<140> log likelihood: -13727
INFO:lda:<150> log likelihood: -13688
INFO:lda:<160> log likelihood: -13716
INFO:lda:<170> log likelihood: -13686
INFO:lda:<180> log likelihood: -13696
INFO:lda:<190> log likelihood: -13700
INFO:lda:<200> log likelihood: -13678
INFO:lda:<210> log likelihood: -13679
INFO:lda:<220> log likelihood: -13611
INFO:lda:<230> log likelihood: -13654
INFO:lda:<240> log likelihood: -13669
INFO:lda:<250> log likelihood: -13618
INFO:lda:<260> log likelihood: -13600
INFO:lda:<270> log likelihood: -13650
INFO:lda:<280> log likelihood: -13656
INFO:lda:<290> log likelihood: -13620
INFO:lda:<300> log likelihood: -13692
INFO:lda:<310> lo

...

Background : 0.2881842864953442
BARRIERS


 31%|███       | 4/13 [01:38<03:34, 23.88s/it]

Didn't find grading for Barriers
BEYOND


INFO:lda:n_documents: 1134
INFO:lda:vocab_size: 3039
INFO:lda:n_words: 19376
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -213911
INFO:lda:<10> log likelihood: -156088
INFO:lda:<20> log likelihood: -152621
INFO:lda:<30> log likelihood: -151237
INFO:lda:<40> log likelihood: -150204
INFO:lda:<50> log likelihood: -149525
INFO:lda:<60> log likelihood: -149242
INFO:lda:<70> log likelihood: -148885
INFO:lda:<80> log likelihood: -148283
INFO:lda:<90> log likelihood: -148059
INFO:lda:<100> log likelihood: -147814
INFO:lda:<110> log likelihood: -148026
INFO:lda:<120> log likelihood: -147683
INFO:lda:<130> log likelihood: -147450
INFO:lda:<140> log likelihood: -147352
INFO:lda:<150> log likelihood: -147061
INFO:lda:<160> log likelihood: -146994
INFO:lda:<170> log likelihood: -147268
INFO:lda:<180> log likelihood: -147021
INFO:lda:<190> log likelihood: -147123
INFO:lda:<200> log likelihood: -146748
INFO:lda:<210> log likelihood: -146738
INFO:lda:<220> log likelihood: -

KeyboardInterrupt: ignored

We noticed 2 errors, and will be rewriting some of the above code to deal with them.

First of all, it doesn't make sense to train across transcripts, because each transcript has its set of annotators, which may not be equivalent each time. We end up considering that irrelevant annotators marked the sentence unannotable.

Instead, we must call EM on each transcript for a given research question. The modification needed is that within the rq loop, we need to collect a list of dataframes (one for each applicable transcript). For each dataframe, run EM by supplying and re-accepting the classifier. At the end, measure accuracy for each rq. We also need to start passing a dictionary of learned parameters.

In [ ]:
def individual_collect_data(rq):
  relevant = [name for name in rq_map if rq in rq_map[name]] ##We got the relevant names
  rel_grad = [t + ".json" for t in relevant if t in graded]
  rel_ungrad = list(set(relevant) - set(rel_grad))
  rel_ungrad = [name + ".json" for name in rel_ungrad]
  train_list = []
  test_list = []
  for rel_doc in rel_ungrad:
    train_data = merge_JsonFiles([rel_doc]) ##We have a list of items. We want to apply clean to each of their text, and if we return an empty string, remove this string.
    if len(rel_grad) == 0:
      return -1, -1
    train_index = 0
    while train_index < len(train_data):
      train_data[train_index]["text"] = preprocess(train_data[train_index]["text"]).strip().replace("  ", " ")
      text = (train_data[train_index]["text"])
      while "  " in text:
        text = text[:text.index("  ")] + text[text.index("  ") + 1: ]
      text = str(text)
      train_data[train_index]["text"] = text
      assert "  " not in train_data[train_index]["text"], train_data[train_index]["text"] 
      if(train_data[train_index]["text"].strip() == ""):
        train_data = train_data[:train_index] + train_data[train_index + 1:]
      else:
        train_index += 1
    train_list.append(train_data)
  
  test_data = merge_JsonFiles(rel_grad)
  test_index = 0
  while test_index < len(test_data):
    test_data[test_index]["text"] = preprocess(test_data[test_index]["text"]).strip()
    text = (test_data[test_index]["text"])
    while "  " in text:
      text = text[:text.index("  ")] + text[text.index("  ") + 1: ]
    test_data[test_index]["text"] = str(text)
    assert "  " not in test_data[test_index]["text"], test_data[test_index]["text"] 
    if(test_data[test_index]["text"].strip() == ""):
      test_data = test_data[:test_index] + test_data[test_index + 1:]
    else:
      test_index+=1

  return train_list, test_data 

Pre-Train Tokenizer and LDA on whole train text. Then use LDA.transform(text) to generate the topic vector.

In [ ]:
def lda_train(rq, train_vectors, test_vectors, train_data, test_data, train_docs, test_docs, tokenizer):
  X = np.array(train_vectors).astype(int)
  vocab = list(tokenizer.word_index.keys())
  model = lda.LDA(n_topics = 10, n_iter = 1500, random_state = 1)
  model.fit(X)

  return model

In [ ]:
def known_tokenize(train_docs, test_docs, tokenizer):
  train_vectors = tokenizer.texts_to_matrix(train_docs, mode='count')
  test_vectors = tokenizer.texts_to_matrix(test_docs, mode='count')
  return train_vectors[:, 1:], test_vectors[:, 1:]

In [ ]:
def lda_df_labels(ldamodel, rq, train_vectors, test_vectors, train_data, test_data, train_docs, test_docs, tokenizer, topics = 10):
  
  X = np.array(train_vectors).astype(int)
  vocab = list(tokenizer.word_index.keys())
  test_vocab = list(tokenizer.word_index.keys())
  
  train_label_dict = dict()
  test_label_dict = dict()
  train_annotators = set()
  test_annotators = set()
  train_df_data = []
  test_df_data = []
  
  ##For each sentence, store the sentence. 
  
  for item in train_data:
    if has_rq(rq, item["annotations"]):
      rq_item = item
      rq_item["annotations"] = [ann for ann in rq_item["annotations"] if ann["ResearchQuestionId"] == rq]
      rq_item["count"] = len(rq_item["annotations"])
      train_df_data.append(rq_item)
    
    else:
      continue

    for annotation in rq_item["annotations"]:
      train_annotators.add(annotation["id"])

  train_annotators = sorted(list(train_annotators))

  for rq_item in train_df_data:
    text = rq_item["text"]
    
    if text not in train_label_dict.keys():
      train_label_dict[text] = []
      for ann in train_annotators:
        present = False
        for itm in rq_item["annotations"]:
          if ann == itm["id"]:
            present = True
        if present:
          train_label_dict[text].append(1.0)
        else:
          train_label_dict[text].append(0.0)
  
  for item in test_data:
    if "grader_annotations" in item.keys():
      test_df_data.append(item)

    for annotation in rq_item["annotations"]:
      test_annotators.add(annotation["id"])

  test_annotators = sorted(list(test_annotators))
  
  for rq_item in test_df_data:
    text = rq_item["text"]
    
    if text not in test_label_dict.keys():

      test_label_dict[text] = []    
      try:
        test_label_dict[text] = (rel_map(rq_item["grader_annotations"][rq]))
      except:
        print("rq: ", rq)
        print("text: ", text)
        print(rq_item["grader_annotations"])
        assert False

  topics_labels_train = defaultdict(list)
  topics_labels_train["text"] = list(train_docs)

  topics_labels_test = defaultdict(list)
  topics_labels_test["text"] = list(test_label_dict.keys())

  for txt in train_docs:
    index = train_docs.index(txt)
    txt_vec = np.array(tokenizer.texts_to_matrix([txt], mode='count'), dtype = np.int64)
    #print("Txt_vec: ", txt_vec)
    distribution = ldamodel.transform(txt_vec[:, 1:])[0]
    for j in range(topics):
    #for j, distribution in enumerate(doc_topic[index]):
      topics_labels_train["topic" + str(j)].append(distribution[j])
  
  for ann_index in range(len(train_annotators)):
    ann = train_annotators[ann_index]
    try:
      topics_labels_train[ann] = [train_label_dict[txt][ann_index] if txt in train_label_dict.keys() else 0.0 for txt in train_docs]
    except:
      for txt in train_docs:
        if txt not in train_label_dict.keys():
          assert False
  
  for txt in test_label_dict.keys():
    index = test_docs.index(txt)
    txt_vec = np.array(tokenizer.texts_to_matrix([txt], mode = 'count'), dtype = np.int64)
    #print("txt_vec:",txt_vec)
    distribution = ldamodel.transform(txt_vec[:, 1:])[0]
    #print(distribution)
    
    for j in range(topics):
    #for j, distribution in enumerate(test_doc_topic[index]):
      topics_labels_test["topic" + str(j)].append(distribution[j])

  #for ann_index in range(len(test_annotators)):
  #  ann = test_annotators[ann_index]
  #  topics_labels_test[ann] = [test_label_dict[txt][ann_index] if txt in test_label_dict.keys() else 0.0 for txt in test_docs]
  
  topics_labels_test["grader"] = [test_label_dict[txt] for txt in test_label_dict.keys()]
  
  ##I don't know how to match the cleaned sentences and the direct transcript that we had before....
  
  
  the_len = -1
  for key in topics_labels_test.keys():
    if the_len == -1:
      the_len = len(topics_labels_test[key])
    assert len(topics_labels_test[key]) == the_len
  
  topics_labels_test_df = pd.DataFrame(topics_labels_test)
  topics_labels_train_df = pd.DataFrame(topics_labels_train)
  return topics_labels_train_df, topics_labels_test_df

In [ ]:
rq_accs = []
rq_trains = []

for rq_index in tqdm(range(len(rqs))):
  rq = rqs[rq_index]
  train_data, test_data = collect_data(rq)
  if(isinstance(train_data, int) or isinstance(test_data, int)):
    continue  
  train_docs, test_docs = text_preprocess(train_data, test_data) ##Will convert to text and preprocess both training and testing data.
  train_vectors, test_vectors, tokenizer, test_tokenizer = tokenize(train_docs, test_docs) ##Will tokenize them and return the corresponding matrices
  ldamodel = lda_train(rq, train_vectors, test_vectors, train_data, test_data, train_docs, test_docs, tokenizer)
  
  train_list, test_data = individual_collect_data(rq) ##Collect Data will merge all JsonFiles with this rq into training data and testing data based on whether there are any grader annotations

  classifier = LogisticRegression()
  skip = False
  for train_data in train_list:
    if(isinstance(train_data, int) or isinstance(test_data, int)):
      print("Didn't find grading for", rq)
      skip = True
      continue
    
    train_docs, test_docs = text_preprocess(train_data, test_data) ##Will convert to text and preprocess both training and testing data.
    
    train_vectors, test_vectors = known_tokenize(train_docs, test_docs, tokenizer) ##Will tokenize them and return the corresponding matrices

    train_df, test_df = lda_df_labels(ldamodel, rq, train_vectors, test_vectors, train_data, test_data, train_docs, test_docs, tokenizer) ##Will fit the train/test vectors independently to topics, and model their distributions, then collect labels, and convert all of them into the finished dataframe. For now, we will exclude the annotator labels though we collect them, to allow LogisticRegression to be tested on them first.

    X_train, majority_train, y_train, X_test, y_test = iosplit(train_df, test_df, aggregate = False) ##Will split each dataframe into input and output for the model

    if list(y_test).count(1) == 0 or list(y_test).count(0) == 0:
      print("Data is imbalanced, skipping.")
      skip = True
      continue
    
    
    majority_train = majority_train.reshape(-1, 1)

    classifier = binary_em(X_train, y_train, majority_train, classifier)
    print("Defined Weights")
    weights = classifier.coef_[0]
    skip = False
    rq_trains.append(f1_score(majority_train, classifier.predict(X_train), average = "macro"))
  
  if(not skip):
    classifier.fit(X_train, majority_train)
    classifier.coef_[0] = weights
    rq_accs.append(f1_score(y_test, classifier.predict(X_test), average = "macro"))
    print(str.capitalize(rq), ":", rq_accs[-1])  

  0%|          | 0/13 [00:00<?, ?it/s]INFO:lda:n_documents: 4261
INFO:lda:vocab_size: 3251
INFO:lda:n_words: 26737
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -292173
INFO:lda:<10> log likelihood: -210040
INFO:lda:<20> log likelihood: -205106
INFO:lda:<30> log likelihood: -203420
INFO:lda:<40> log likelihood: -202216
INFO:lda:<50> log likelihood: -201446
INFO:lda:<60> log likelihood: -200826
INFO:lda:<70> log likelihood: -200270
INFO:lda:<80> log likelihood: -199750
INFO:lda:<90> log likelihood: -199323
INFO:lda:<100> log likelihood: -198941
INFO:lda:<110> log likelihood: -198850
INFO:lda:<120> log likelihood: -198589
INFO:lda:<130> log likelihood: -198504
INFO:lda:<140> log likelihood: -198232
INFO:lda:<150> log likelihood: -197931
INFO:lda:<160> log likelihood: -198006
INFO:lda:<170> log likelihood: -197617
INFO:lda:<180> log likelihood: -197666
INFO:lda:<190> log likelihood: -197466
INFO:lda:<200> log likelihood: -197800
INFO:lda:<210> log likelihood: -1

X_train.shape:  (130, 10)
y_train.shape:  (130, 22)
stat_out.shape:  (130,)
X_test.shape:  (118, 10)
y_test.shape:  (118, 1)
Data is imbalanced, skipping.


X_train.shape:  (164, 10)
y_train.shape:  (164, 23)
stat_out.shape:  (164,)
X_test.shape:  (118, 10)
y_test.shape:  (118, 1)
Data is imbalanced, skipping.


X_train.shape:  (339, 10)
y_train.shape:  (339, 25)
stat_out.shape:  (339,)
X_test.shape:  (118, 10)
y_test.shape:  (118, 1)
Data is imbalanced, skipping.


X_train.shape:  (334, 10)
y_train.shape:  (334, 26)
stat_out.shape:  (334,)
X_test.shape:  (118, 10)
y_test.shape:  (118, 1)
Data is imbalanced, skipping.


X_train.shape:  (216, 10)
y_train.shape:  (216, 22)
stat_out.shape:  (216,)
X_test.shape:  (118, 10)
y_test.shape:  (118, 1)
Data is imbalanced, skipping.


X_train.shape:  (75, 10)
y_train.shape:  (75, 7)
stat_out.shape:  (75,)
X_test.shape:  (118, 10)
y_test.shape:  (118, 1)
Data is imbalanced, skipping.


  0%|          | 0/13 [00:23<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
rq_accs, rq_train

In [ ]:
from numpy.core.numeric import binary_repr
rq_accs = []
rq_trains = []
for rq_index in tqdm(range(len(rqs))): 
  rq = rqs[rq_index]
  print(rq.upper())

  train_list, test_data = collect_data(rq) ##Collect Data will merge all JsonFiles with this rq into training data and testing data based on whether there are any grader annotations

  classifier = LogisticRegression()

  for train_data in train_list:
    if(isinstance(train_data, int) and isinstance(test_data, int)):
      print("Didn't find grading for", rq)
      continue
    
    train_docs, test_docs = text_preprocess(train_data, test_data) ##Will convert to text and preprocess both training and testing data.
    
    train_vectors, test_vectors, tokenizer = tokenize(train_docs, test_docs) ##Will tokenize them and return the corresponding matrices

    train_df, test_df = lda_df_labels(rq, train_vectors, test_vectors, train_data, test_data, train_docs, test_docs, tokenizer) ##Will fit the train/test vectors independently to topics, and model their distributions, then collect labels, and convert all of them into the finished dataframe. For now, we will exclude the annotator labels though we collect them, to allow LogisticRegression to be tested on them first.

    X_train, majority_train, y_train, X_test, y_test = iosplit(train_df, test_df, aggregate = False) ##Will split each dataframe into input and output for the model

    if list(y_test).count(1) == 0 or list(y_test).count(0) == 0:
      print("Data is imbalanced, skipping.")
      continue
    
    
    majority_train = majority_train.reshape(-1, 1)

    classifier = binary_em(X_train, y_train, majority_train, classifier)
    print("Defined Weights")
    weights = classifier.coef_[0]
    rq_trains.append(f1_score(majority_train, classifier.predict(X_train), average = "macro"))
  
  classifier.fit(X_train, majority_train)
  classifier.coef_[0] = weights
  rq_accs.append(f1_score(y_test, classifier.predict(X_test), average = "macro"))
  print(str.capitalize(rq), ":", rq_accs[-1])

In [ ]:
import tqdm
from tqdm import tqdm
from numpy.linalg import inv
import numba

def binary_em(X_train, y_train, majority_train, classifier):
  alpha = np.sum(np.matmul(y_train.T, majority_train), axis = 0)/np.sum(majority_train)
  beta = np.sum(np.matmul(1 - y_train.T, 1 - majority_train), axis = 0)/np.sum(1-majority_train)
  mu = np.mean(y_train, axis = 0)
  a = np.prod(np.power(alpha, y_train), axis = 1) * np.prod(np.power(1 - alpha, 1 - y_train), axis = 1) 
  b = np.prod(np.power(beta, 1 - y_train), axis = 1) * np.prod(np.power(1 - beta, y_train), axis = 1)
  classifier.fit(X_train, majority_train)
  p = classifier.predict_proba(X_train)[:, 0]
  em_steps = 15
  grad_steps = 300
  lr = 0.01
  for e in tqdm(range(em_steps)):
    
    ##Expectation:
    a = np.prod(np.power(alpha, y_train), axis = 1) * np.prod(np.power(1 - alpha, 1 - y_train), axis = 1) 
    b = np.prod(np.power(beta, 1 - y_train), axis = 1) * np.prod(np.power(1 - beta, y_train), axis = 1)
    p = classifier.predict_proba(X_train)[:,0]
    mu = a * p / (a * p + b * (1 - p))
    E = np.sum(mu * np.log(p*a) + (1 - mu) * np.log((1 - p) * b))
    
    ##Maximization:

    alpha = np.matmul(y_train.T, mu).T / np.sum(mu)
    beta = np.matmul((1 - y_train.T), (1 - mu)).T / np.sum(1 - mu)

    ##Learning Weights:

    weights = classifier.coef_[0]
    for grad_steps in range(grad_steps):
      pred = classifier.predict_proba(X_train)[:, 0]
      grad = np.sum(np.multiply((mu - pred).reshape(-1,1), X_train), axis = 0)
      
      #Gradient Descent:
      ##weights -= lr * grad
      
      ##Newton's Method:
      hess_shape = (np.shape(X_train))[1]
      hess_shape = (hess_shape, hess_shape)
      hessian = np.zeros(hess_shape)
      for i in range(len(X_train)):
          hessian -= pred[i] * (1 - pred[i]) * np.outer(X_train[i], X_train[i])
      hessian_inv = inv(hessian)
      weights = weights - lr * np.matmul(hessian_inv, grad)
      classifier.coef_[0] = weights
  
  return classifier, alpha, beta